# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.test_framework.key import generate_bip340_key_pair
from functions.test_framework.address import program_to_witness
from functions.bip0327.reference import individual_pk, key_agg_and_tweak, get_xonly_pk, key_sort

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-09-06T01:22:58.048000Z TestFramework (INFO): PRNG seed is: 5451303301215914080
2025-09-06T01:22:58.050000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_nnrbvdp1


### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
alice_funding_pubkey_b = alice_funding_pubkey.get_bytes()
alice_funding_privkey_b = alice_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey_b)
print(f"Alice funding pubkey: {alice_funding_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_funding_privkey_b.hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
bob_funding_pubkey_b = bob_funding_pubkey.get_bytes()
bob_funding_privkey_b = bob_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey_b)
print(f"Bob funding pubkey: {bob_funding_pubkey_b.hex()}")
print(f"Bob funding privkey: {bob_funding_privkey_b.hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
print(f"Transaction creating Alice UTXO: {txid_to_spend}")
rawtx_json = node.getrawtransaction(txid_to_spend, True)
print(json.dumps(rawtx_json, indent=2, default=str))

expected_spk = (b"\x51" + b"\x20" + alice_funding_pubkey_b).hex()
index_to_spend = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == expected_spk
)
print(f"Alice transaction to spend: {txid_to_spend}:{index_to_spend}")

result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: 47e1373fe2da4974cfc96af84ffc7581f4744608b42180d50c1fbb221ba74626
Alice funding privkey: 2463a6fdbe1db46d5e7ad31db5b1c6a0e6fe01c97ccfc601809536cfcd72b6a2
Alice funding address: bcrt1pglsnw0lzmfyhfn7fdtuyllr4s868g3sgksscp4gvr7ajyxa8gcnqc927wp
Bob funding pubkey: 512af71fe3a44d8b00624cba05ee2b79e1cdd076a3be2c1e6c35dc17db2e0552
Bob funding privkey: 65d30794eb0333908012606befb1592a06f1fa9bbaef88241435198967b07664
Bob funding address: bcrt1p2y40w8lr53xckqrzfjaqtm3t08sum5rk5wlzc8nvxhwp0kewq4fqsuxmm4
Transaction creating Alice UTXO: d9ba0b9d00d4e03a5c60ff01092cc189a72bfe3bf2649e612d105639bba2ca0e
{
  "txid": "d9ba0b9d00d4e03a5c60ff01092cc189a72bfe3bf2649e612d105639bba2ca0e",
  "hash": "1ee0f56ddb51d99097d39a3f63af8f420acb2271b72d9dfe77c5d3b43338689e",
  "version": 2,
  "size": 289,
  "vsize": 208,
  "weight": 829,
  "locktime": 97,
  "vin": [
    {
      "txid": "6c5b31b24cc8c49bbfe33151b5e882fd75d63d8f14c75a8f074d44d404c478ea",
      "vout": 0,
      "scriptSig": {
     

## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()
alice_change_pubkey_b = alice_change_pubkey.get_bytes()
alice_change_privkey_b = alice_change_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey_b)
print(f"Alice funding pubkey: {alice_change_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_change_privkey_b.hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: 7b830785b2a997801c85bd24cc0df41c5d7d2abf99ed6bac3aed5263d0702ed1
Alice funding privkey: 1c6daaecc3daf1308a164f4bbad667b7a1de70d1af21ebae580d4d38e95ec18e
Alice funding address: bcrt1p0wps0pdj4xtcq8y9h5jvcr05r3wh624ln8kkhtp6a4fx85rs9mgsgckjjl


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding musig2 output
privkey_alice_musig2, _ = generate_bip340_key_pair()
pubkey_alice_musig2 = individual_pk(privkey_alice_musig2.get_bytes())
print(f"Alice MuSig2 pubkey: {pubkey_alice_musig2.hex()}")
print(f"Alice MuSig2 privkey: {privkey_alice_musig2.get_bytes().hex()}")

# Create a new Bob key pair for channel funding musig2 output
privkey_bob_musig2, _ = generate_bip340_key_pair()
pubkey_bob_musig2 = individual_pk(privkey_bob_musig2.get_bytes())
print(f"Bob MuSig2 pubkey: {pubkey_bob_musig2.hex()}")
print(f"Bob MuSig2 privkey: {privkey_bob_musig2.get_bytes().hex()}")

tag = "TapTweak"
tweak = sha256(tag.encode('utf-8'))

# Generate a 2-of-2 tweaked aggregate MuSig2 key using the pubkeys form Alice and Bob
sorted_pubkeys = key_sort([pubkey_alice_musig2, pubkey_bob_musig2])
key_agg_ctx = key_agg_and_tweak(sorted_pubkeys, [tweak], [True])
agg_pubkey = get_xonly_pk(key_agg_ctx)

print(f"Aggregate musig2 pubkey: {agg_pubkey.hex()}")

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey)
print(f"Channel funding musig2 output address: {channel_funding_musig_address}")

Alice MuSig2 pubkey: 020f94b0cccd67b02cbf730dc5c2f9fc61af5a2903b88dd004241fdd29bc7b769f
Alice MuSig2 privkey: 281949a6e965c6e7eab1776ff82c977c62e59d9b640fe8c342250fc6fbe861e9
Bob MuSig2 pubkey: 02c2463d118f5b90c3dc98f67017138701a1abf9e567f7b15a33260005f8fe7142
Bob MuSig2 privkey: eefb431c72864463212a26304f8d7a50e9922e89d79043e611936e7cdfa290ca
Aggregate musig2 pubkey: 34aa42b3162cea1677dc662d43f9e27220926ba57fc3b93405efe48c84eb0645
Channel funding musig2 output address: bcrt1pxj4y9vck9n4pva7uvck5870zwgsfy6a90lpmjdq9aljgep8tqezsnazxtg


#### The channel funding transaction

##### The unsigned transaction

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
output1_value_sat = int(float("0.01") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(agg_pubkey) + agg_pubkey

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
output2_value_sat = int(float(alice_initial_fund) * 100000000) - output1_value_sat - tx_fee_sat
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey_b) + alice_change_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001b6175c02b1d9c64596f7f74d9a06a6729841f39712238e09fd545df2832ee1fc0200000000ffffffff0240420f00000000002251206f79d24b23c743fdef3476e2fbc96a94470932eb491ffad0150919c9036a10d7949de60500000000225120109a7e641ada76ba8bf1af9ec47115af3124b9ef72e314bca055e612577915bc00000000
{
  "txid": "69340c9cd58cf9c91e5f3ef98c9e9ce972e760f0101925f9c31190288b35b3d0",
  "hash": "69340c9cd58cf9c91e5f3ef98c9e9ce972e760f0101925f9c31190288b35b3d0",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "fce12e83f25d54fd098e231297f3419872a6069a4df7f79645c6d9b1025c17b6",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 6f79d24b23c743fdef3476e2fbc96a94470932eb491ffad0150919c9036a10d7",
        "desc": "rawtr(6f79d24b23c743fdef3476e2fbc96a94470932eb491ffad0150919c9

##### The sighash for key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [7]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


##### Signing the sighash

In [8]:
aux_rand = b"\x00"*32
signature = alice_funding_privkey.sign_schnorr(sighash, aux_rand) 

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

0140dc544aeefd7f0dcff553239511205ffab50b275fcd830f088cdcf0207e48a339e1f9789ea5b48fe0d9f3da30131488d90d49dea49a2af26abcfe58c299617c75
signed tx:  02000000000101b6175c02b1d9c64596f7f74d9a06a6729841f39712238e09fd545df2832ee1fc0200000000ffffffff0240420f00000000002251206f79d24b23c743fdef3476e2fbc96a94470932eb491ffad0150919c9036a10d7949de60500000000225120109a7e641ada76ba8bf1af9ec47115af3124b9ef72e314bca055e612577915bc0140dc544aeefd7f0dcff553239511205ffab50b275fcd830f088cdcf0207e48a339e1f9789ea5b48fe0d9f3da30131488d90d49dea49a2af26abcfe58c299617c7500000000
{
  "txid": "69340c9cd58cf9c91e5f3ef98c9e9ce972e760f0101925f9c31190288b35b3d0",
  "hash": "45268e6942a83bac54c3f7a2076f980fba3844d20299c5589babd4fc1a2cb60e",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "fce12e83f25d54fd098e231297f3419872a6069a4df7f79645c6d9b1025c17b6",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [